In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

In [15]:
df = pd.read_csv("/content/train_Titanic.csv")
df_test = pd.read_csv("/content/test_Titanic.csv")

In [16]:
def i(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

def j(tt):
    if tt in ['Mr']:
        return 1
    elif tt in ['Master']:
        return 3
    elif tt in ['Ms', 'Mlle', 'Miss']:
        return 4
    elif tt in ['Mrs','Mme']:
        return 5
    else:
        return 2

df['title'] = df['Name'].apply(i).apply(j)
df_test['title'] = df_test['Name'].apply(i).apply(j)

df = df.drop(['PassengerId','Name','Ticket'], axis=1)
df_test = df_test.drop(['Name','Ticket'], axis=1)

edt  = pd.get_dummies(df['Embarked'])
edt.drop(['S'], axis=1, inplace=True)
edt_test  = pd.get_dummies(df_test['Embarked'])
edt_test.drop(['S'], axis=1, inplace=True)
df = df.join(edt)
df_test = df_test.join(edt_test)
df.drop(['Embarked'], axis=1,inplace=True)
df_test.drop(['Embarked'], axis=1,inplace=True)

df_test["Fare"].fillna(df_test["Fare"].median(), inplace=True)

df['Age'] = df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))
df_test['Age'] = df_test.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

df.drop("Cabin",axis=1,inplace=True)
df_test.drop("Cabin",axis=1,inplace=True)

s = sorted(df['Sex'].unique())
z=zip(s, range(0, len(s) + 1))
gm = dict(z)
df['Sex'].head(5)
df['Sex'] = df['Sex'].map(gm).astype(int)
df['Sex'].head(5)
df_test['Sex'] = df_test['Sex'].map(gm).astype(int)

In [17]:
X_train = df.drop("Survived",axis=1)
Y_train = df["Survived"]

In [18]:
model =  LogisticRegression()

In [19]:
%%capture
model.fit(X_train, Y_train)

In [20]:
model.score(X_train, Y_train)

0.8249158249158249

In [21]:
X_test  = df_test.drop("PassengerId",axis=1).copy()

In [22]:
Y_pred = model.predict(X_test)

In [23]:
dfo = pd.DataFrame({ "Id": df_test["PassengerId"], "Survived": Y_pred })

In [24]:
dfo.to_csv('/content/out_Titanic.csv',index=False)